In [3]:
import gurobipy as gp
def solve_linear_problem(filename):
  # Read the problem data from a file
  problem = gp.read(filename)

  # Create a Gurobi model instance
  model = gp.Model()

  # Add the decision variables to the model
  variables = {}
  for var_name in problem.getVars():
      var = model.addVar(lb=var_name.LB, ub=var_name.UB, obj=var_name.Obj, name=var_name.VarName)
      variables[var_name.VarName] = var

  # Add the constraints to the model
  for constr_name in problem.getConstrs():
      constr_expr = problem.getRow(constr_name)
      lhs = gp.LinExpr()
      for i in range(constr_expr.size()):
          lhs += variables[constr_expr.getVar(i).VarName] * constr_expr.getCoeff(i)
      constr = model.addConstr(lhs=lhs, sense=constr_name.Sense, rhs=constr_name.RHS)

  # Set the objective function
  model.setObjective(gp.quicksum(variables[var_name.VarName] * var_name.Obj for var_name in problem.getVars()), problem.ModelSense)

  # Optimize the model
  model.optimize()
  point = []
  # Print the optimal objective value and variable values
  print("##################solution##################")
  print('Optimal objective value:', model.objVal)
  for var_name in problem.getVars():
      point.append(variables[var_name.VarName].X)
      print(var_name.VarName, variables[var_name.VarName].X)
  print(point)
  print("############################################")
  return point

In [ ]:
solve_linear_problem('problem.lp')

Read LP format model from file problem.lp
Reading time = 0.00 seconds
: 2 rows, 2 columns, 3 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 2 columns and 3 nonzeros
Model fingerprint: 0xa7dd2a6a
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+00
##################solution##################
Optimal objective value: 1.0
x 1.0
y 0.0
{'x': 1.0, 'y': 0.0}
##############

<ipython-input-64-034752c717ba>:1: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  solve_linear_problem('problem.lp')


{'x': 1.0, 'y': 0.0}

In [2]:
!pip install gurobipy==10.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 27.1 MB/s eta 0:00:00


In [4]:
import sympy as sp

def linearize(func, variables, point):
    """
    Linearizes a nonlinear function using first-order approximation
    Args:
        func (sp.Basic): Nonlinear function to be linearized
        variables (list): List of variables to be linearized
        point (dict): Dictionary of values representing the point of linearization
    Returns:
        sp.Expr: Linearized function
    """
    # Create a symbolic expression for the function
    func_expr = sp.sympify(func)

    # Create symbolic variables for the point of linearization
    point_vars = {var: sp.Symbol(var) for var in variables}

    # Replace variables in the function with the symbolic variables
    func_expr = func_expr.subs(point_vars)

    # Compute the first-order partial derivatives
    derivatives = [sp.diff(func_expr, var) for var in variables]

    # Evaluate the derivatives at the point of linearization
    derivatives_at_point = [derivative.subs(point) for derivative in derivatives]

    # Create the linearized function using first-order approximation
    linearized_func = func_expr.subs(zip(variables, point.values()))
    for i in range(len(variables)):
        linearized_func += derivatives_at_point[i] * (sp.Symbol(variables[i]) - point[variables[i]])

    return linearized_func


In [5]:
import math
import re

def evaluate_expression(expression):
    # Define the regular expression pattern to match the function substitutions
    pattern = r'(?<![a-zA-Z0-9_])((sin|cos|tan|tanh|exp|log|sqrt)\((-?\d+(\.\d+)?)\))(?![a-zA-Z0-9_])'

    # Define a function to evaluate the matched substitutions
    def evaluate(match):
        function = match.group(2)
        number = float(match.group(3))
        result = expression
        if function == 'sin':
            return str(math.sin(number))
        elif function == 'cos':
            return str(math.cos(number))
        elif function == 'tan':
            return str(math.tan(number))
        elif function == 'tanh':
            return str(math.tanh(number))
        elif function == 'exp':
            return str(math.exp(number))
        elif function == 'log':
            return str(math.log(number))
        elif function == 'sqrt':
            return str(math.sqrt(number))

    # Perform the substitutions using the evaluate function
    result = expression
    match = re.search(pattern, str(expression))
    print(match)
    if match:
      result = re.sub(pattern, evaluate, str(expression))

    return result


In [ ]:
# import sympy as sp

# def linearize_expression(expression, variables, center):
#     """
#     Linearize a non-linear expression with multiple variables using a Taylor series expansion.

#     Arguments:
#     expression -- The non-linear expression to be linearized.
#     variables -- The variables of the expression.
#     center -- The point around which the Taylor series expansion is performed.

#     Returns:
#     The linearized expression.
#     """
#     # Convert the expression from a string to a SymPy expression
#     expression = sp.sympify(expression)

#     # Compute the partial derivatives of the expression with respect to each variable
#     derivatives = [expression.diff(variable) for variable in variables]

#     # Compute the linearized expression using the partial derivatives and the center point
#     linearized_expression = expression.subs([(variable, value) for variable, value in zip(variables, center)])

#     for variable, derivative in zip(variables, derivatives):
#         variable_value = center[variables.index(variable)]
#         linearized_expression += derivative.subs([(variable, variable_value)]) * (variable - variable_value)

#     linearized_expression = evaluate_expression(linearized_expression)
#     return linearized_expression


In [6]:
import sympy as sp

def linearize_expression(expression, variables, center):
    """
    Linearize a non-linear expression with multiple variables using a Taylor series expansion.

    Arguments:
    expression -- The non-linear expression to be linearized.
    variables -- The variables of the expression.
    center -- The point around which the Taylor series expansion is performed.

    Returns:
    The linearized expression.
    """
    # Convert the expression from a string to a SymPy expression
    expression = sp.sympify(expression)

    # Compute the partial derivatives of the expression with respect to each variable
    derivatives = [expression.diff(variable) for variable in variables]

    # Compute the linearized expression using the partial derivatives and the center point
    linearized_expression = expression.subs([(variable, value) for variable, value in zip(variables, center)])

    for variable, derivative in zip(variables, derivatives):
        variable_value = center[variables.index(variable)]
        linearized_expression += derivative.subs([(var, val) for var, val in zip(variables, center)]) * (variable - variable_value)

    linearized_expression = sp.simplify(linearized_expression)
    linearized_expression = evaluate_expression(linearized_expression)
    return linearized_expression


In [168]:
def convert_division_to_multiplication(expression):
    terms = re.split(r'([-+*/])', expression)
    converted_terms = []

    i = 0
    while i < len(terms):
        term = terms[i]
        if term.strip() == '/':
            divisor = float(terms[i + 1])
            variable = terms[i + 2].strip()
            converted_terms.append(f'{divisor:.15g} * {variable}')
            i += 2
        else:
            converted_terms.append(term)
        i += 1

    converted_expression = ''.join(converted_terms)
    return converted_expression

In [150]:
from sympy import symbols, sympify, Float

def simplify_expression(expression):
    # Parse the expression
    parsed_expr = sympify(expression)

    # Get the individual terms
    terms = parsed_expr.as_ordered_terms()

    # Separate the terms into constant and variable parts
    constant_terms = []
    variable_terms = []
    for term in terms:
        if term.is_number:
            constant_terms.append(term)
        else:
            variable_terms.append(term)

    # Combine the constant and variable parts
    simplified_str = " + ".join(str(term) for term in variable_terms)
    if constant_terms:
        constant_str = " + ".join(str(Float(term)) for term in constant_terms)
        if simplified_str:
            simplified_str += " + " + constant_str
        else:
            simplified_str = constant_str

    return simplified_str

In [169]:
ex1 = simplify_expression("x/2 + 5/2 - y/2 - x")
print(ex1)
ex2 = convert_division_to_multiplication(ex1)
print(ex2)

-x/2 + -y/2 + 2.50000000000000
-x2 * + -y2 * + 2.50000000000000


In [8]:
# Example usage:
expression = '(x - 4)*2.1 + 3*(y - 5.5) - 5 + 7'
rearranged = simplify_expression(expression)
print(rearranged)


2.1*x + 3*y - 22.9


In [9]:
def read_obj_func(filename):
    variables = []
    objective_function = ""
    is_max = False
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith("VAR"):
                # Extract variable names
                variable_line = line.split(' ')
                var = variable_line[1:]
                var = sp.symbols(var)
                variables.extend(var)
            elif line.startswith("MAX") or line.startswith("MIN"):
                if line.startswith("MAX"):
                    is_max =True
                # Extract objective function
                objective_function = line[4:]

    return variables, objective_function, is_max

In [10]:
from sympy.core.evalf import N
import re

def remove_trailing_operators(expression):

    # Use regular expressions to match the trailing "+" or "-" signs
    pattern = r"([+-])$"
    match = re.search(pattern, expression)

    if match:
        # Remove the trailing operator if found
        expression = expression[:-1]

    return expression

def extract_constant(expression):
    expression = expression.strip()  # Remove leading/trailing whitespaces
    pattern = r'(?<!\*)[-+]?\b\d*\.?\d+(?=\D*$)(?![\w*])'  # Updated pattern to match the constant

    match = re.search(pattern, expression)
    if match:
        extracted_part = match.group(0)
        return extracted_part

    return None

def is_string_number(string):
    try:
        float(string)  # or int(string) if you want to check for integers only
        return True
    except ValueError:
        return False



def remove_constants(expression):
    # # Regular expression pattern to match constant numbers
    # pattern = r'(?<!\*)\b\d+\b(?!\*)'
    # constant = extract_pattern(expression)
    # # Remove constants that are not multiplied with variables
    # expression = re.sub(pattern, '', expression)

    expression = expression.strip()  # Remove leading/trailing whitespaces
    constant_pattern = r"\b[-+]?[0-9]*\.?[0-9]+\b"  # Pattern to match a constant
    constant = extract_constant(expression)

    # Check if the expression ends with a constant
    if re.search(constant_pattern + r"\s*$", expression):
        # Remove the constant from the end of the expression
        expression = re.sub(constant_pattern + r"\s*$", "", expression)

    letters= expression.split(' ')
    is_plus = False
    exit = False
    if constant == None:
      return expression , constant , is_plus , exit
    if len(letters) == 1:
      exit = True
      return expression , constant , is_plus , exit
    elif letters[-2] == '+' :
      is_plus = True
    expression = remove_trailing_operators(expression.strip())
    return expression , constant , is_plus , exit

In [43]:
expression  = 'x/200'
expression , constant , is_plus , exit= remove_constants(expression)
print(expression , constant , is_plus)

x/ 200 False


In [11]:
import re

In [12]:
def extract_function_from_line(equation):
    comparison_operators = ['<', '>', '<=', '>=', '==', '!=']

    for operator in comparison_operators:
        if operator in equation:
            return equation.split(operator)[0].strip()

def extract_inequation(line):
    # Regular expression pattern to match the inequation part
    pattern = r'^c\d+\s+(.*)$'
    match = re.match(pattern, line)

    if match:
        inequation = match.group(1)
        # Extract the comparison operator and the number after it
        operator_and_number = re.findall(r'([<>]=?)\s*(-?\d+)', inequation)
        if operator_and_number:
            operator, number = operator_and_number[0]
            return f"{operator} {number}"

    return None

def remove_first_word(line):
    pattern = r'^c\d+\s(.*)$'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return line

def read_constraints(filename):
    variables = []
    constraints_functions = []
    inequalities = []
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith("VAR"):
                # Extract variable names
                variable_line = line.split(' ')
                var = variable_line[1:]
                var = sp.symbols(var)
                variables.extend(var)
            elif line.startswith("c"):
                # Extract objective function
                inequalities.append(extract_inequation(line))
                line = remove_first_word(line)
                constraints_functions.append(extract_function_from_line(line))

    return variables, constraints_functions , inequalities

In [13]:
def generate_lp_file(objective_function, constraints, filename,inequalities , is_max):
    # Open the file for writing
    with open(filename, 'w') as file:
        # Write the objective function
        if is_max == True :
          file.write("Maximize\n")
        else:
          file.write("Minimize\n")
        file.write(str(objective_function).replace("*", " ") + "\n" + "\n")

        # Write the constraints
        file.write("Subject To\n")
        c = 0
        for constraint in constraints:
            file.write(str(constraint).replace("*", " ") +" " + inequalities[c] + "\n")
            c+=1

        # Write the end of the file
        file.write("\nEnd")

In [14]:
variables, objective_func , is_max = read_obj_func('non_linear.txt')
point = {'x' : 15 , 'y' : 5}
print(objective_func)
vars = []
for var in variables:
  print(var)
  vars.append(sp.symbols(str(var)))
x, y = sp.symbols('x y')
objective_func = linearize_expression(objective_func, vars , [15, 5])
print(objective_func)

sin(x) +sin(y)
x
y
<re.Match object; span=(9, 16), match='cos(15)'>
(x - 15)*-0.7596879128588213 + (y - 5)*0.28366218546322625 + -0.9589242746631385 + 0.6502878401571168


In [16]:
x, y = sp.symbols('x y')
expression = 'sin(x) + cos(y)'

linearized_expr = linearize_expression(expression, [x, y], [1, 0])

print(linearized_expr)

<re.Match object; span=(8, 14), match='cos(1)'>
(x - 1)*0.5403023058681398 + 0.8414709848078965 + 1


In [17]:
variables, constraints_functions , inequalities = read_constraints('non_linear.txt')
constraints_functions[0] = linearize(constraints_functions[0], variables, point)
constraints_functions[1] = linearize(constraints_functions[1], variables, point)

TypeError: ignored

In [ ]:
print(objective_func,constraints_functions,inequalities)

cosx + cosy [0.129099444873581*x + 1.93649167310371, x + y] ['> 1', '>= 0']


In [ ]:
point = {'x' : 1 , 'y' : 8}
objective_func = linearize(objective_func, variables, point)
objective_func , constant , is_plus= remove_constants(str(objective_func))

['203*x', '+', 'y'] y


In [ ]:
print(objective_func,constraints_functions,inequalities)

203*x + y [2*x + 100*y - 200, x + y] ['> 0', '>= 0']


In [ ]:
generate_lp_file(objective_func, constraints_functions, 'problem.lp',inequalities , is_max)

In [ ]:
solve_linear_problem('problem.lp')

Error reading LP format file problem.lp at line 1
Malformed term in expression
Neighboring tokens: " - ) sin() - (y - ) sin() + cos() "

Unable to read file


GurobiError: ignored

In [ ]:
#pseudo code
#1. read_obj_func('non_linear.txt')
#2. read_constraints('non_linear.txt')
#3. prev_point = {'x' : 1 , 'y' : 2} #initilaization
#4. while (|prev_point - new_point| > epsilon)
       # prev_point = new_point
       #linearize(objective_func, variables, point)
       #for contrraint in constraints:
        #linearize(contrraint, variables, point)
       #generate_lp_file(objective_func, constraints_functions, 'problem.lp',inequalities , is_max)
       #new_point = solve_linear_problem('problem.lp')

#5. return new_point

In [18]:
import numpy as np

In [41]:
def solve_project(file):
  variables, objective_func , is_max = read_obj_func(file)
  variables, constraints_functions , inequalities = read_constraints(file)
  print(objective_func,is_max,variables, constraints_functions , inequalities)
  prev_point = [100] #randomly chosen
  new_point = [15]
  new_constraints_functions = []
  new_inequalities = []
  diff = []
  constant_inequalities = []
  for j,var in enumerate(variables) :
    diff.append( new_point[j]-prev_point[j] )
  for inequality in inequalities:
    list_inequality = inequality.split(' ')
    constant_inequalities.append(list_inequality[-1])
  print(np.linalg.norm(diff))
  points = []
  while(np.linalg.norm(diff)>0.1):
    prev_point=new_point
    lin_objective_func = linearize_expression(objective_func, variables , new_point)
    print(lin_objective_func)
    lin_objective_func = simplify_expression(lin_objective_func)
    print(lin_objective_func)
    lin_objective_func , zevel1,zevel2,zevel3  = remove_constants(str(lin_objective_func))
    print(lin_objective_func)
    for i in range(len(constraints_functions)):
      new_const = linearize_expression(constraints_functions[i], variables, new_point)
      new_const = simplify_expression(new_const)
      new_constraints_functions.append(new_const)
      new_constraints_functions[i]  ,constant , is_plus , exit =remove_constants(str(new_constraints_functions[i]))
      if exit :
        print('exitexitexitexitexit')
        print(new_point)
        return
      if constant != None :
          if is_plus:
            new_number =  float(constant_inequalities[i]) - float(constant)
            new_inequalities.append(inequalities[i])
            new_inequalities[i]=new_inequalities[i].replace(constant_inequalities[i],str(new_number))
          else:
            new_number =  float(constant_inequalities[i]) + float(constant)
            new_inequalities.append(inequalities[i])
            new_inequalities[i]=(new_inequalities[i]).replace(constant_inequalities[i],str(new_number))
      else:
        new_inequalities.append(inequalities[i])
    print(lin_objective_func,is_max,variables, new_constraints_functions , new_inequalities)
    generate_lp_file(lin_objective_func, new_constraints_functions, 'problem.lp',new_inequalities , is_max)
    new_point = solve_linear_problem('problem.lp')
    new_constraints_functions = []
    new_inequalities = []
    diff = []
    for j,var in enumerate(variables) :
        diff.append( new_point[j]-prev_point[j] )
    print(np.linalg.norm(diff))
    if new_point in points:
      break
    points.append(new_point)
  print(new_point)
  return

In [115]:
solve_project('non_linear.txt')

x^-1 True [x] ['x'] ['>= 1']
85.0
None
2/15 - x/225
-x/225 + 0.13333333333333333
-x/225 
None
-x/225  True [x] ['x'] ['>= 1']
Read LP format model from file problem.lp
Reading time = 0.00 seconds
: 1 rows, 2 columns, 1 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 2 columns and 1 nonzeros
Model fingerprint: 0x160ccd87
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -0.000000000

<ipython-input-41-a1a3ac4e87b7>:48: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  new_point = solve_linear_problem('problem.lp')


GurobiError: ignored